<a href="https://colab.research.google.com/github/ApoorvBrooklyn/Edu-vitality/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# Define the number of samples per learning type
num_samples = 10000

# Simulate data for different learning types
def generate_data(num_samples, mean_times, mean_accuracies, label):
    response_times_audio = np.random.normal(loc=mean_times[0], scale=2, size=num_samples)
    response_times_picture = np.random.normal(loc=mean_times[1], scale=2, size=num_samples)
    response_times_paragraph = np.random.normal(loc=mean_times[2], scale=2, size=num_samples)

    accuracies_audio = np.random.normal(loc=mean_accuracies[0], scale=0.1, size=num_samples)
    accuracies_picture = np.random.normal(loc=mean_accuracies[1], scale=0.1, size=num_samples)
    accuracies_paragraph = np.random.normal(loc=mean_accuracies[2], scale=0.1, size=num_samples)

    labels = [label] * num_samples

    return pd.DataFrame({
        'response_time_audio': response_times_audio,
        'response_time_picture': response_times_picture,
        'response_time_paragraph': response_times_paragraph,
        'accuracy_audio': accuracies_audio,
        'accuracy_picture': accuracies_picture,
        'accuracy_paragraph': accuracies_paragraph,
        'label': labels
    })

# Generate data for each type of learner
kinesthetic = generate_data(num_samples, mean_times=[8, 10, 12], mean_accuracies=[0.8, 0.7, 0.6], label='Kinesthetic Learner')
auditory = generate_data(num_samples, mean_times=[15, 17, 20], mean_accuracies=[0.7, 0.6, 0.8], label='Auditory Learner')
visual = generate_data(num_samples, mean_times=[20, 18, 22], mean_accuracies=[0.9, 0.85, 0.9], label='Visual Learner')
reading_writing = generate_data(num_samples, mean_times=[10, 15, 8], mean_accuracies=[0.7, 0.75, 0.8], label='Reading/Writing Learner')
logical = generate_data(num_samples, mean_times=[12, 18, 15], mean_accuracies=[0.85, 0.8, 0.9], label='Logical Learner')
social = generate_data(num_samples, mean_times=[18, 12, 15], mean_accuracies=[0.6, 0.7, 0.65], label='Social Learner')
solitary = generate_data(num_samples, mean_times=[22, 20, 25], mean_accuracies=[0.75, 0.7, 0.8], label='Solitary Learner')

# Combine the data into a single DataFrame
data = pd.concat([kinesthetic, auditory, visual, reading_writing, logical, social, solitary], ignore_index=True)

# Shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

# Save dataset to a CSV file for use in training
data.to_csv('learning_preferences.csv', index=False)

In [2]:
data.head()

,response_time_audio,response_time_picture,response_time_paragraph,accuracy_audio,accuracy_picture,accuracy_paragraph,label
0,20.950447,15.187713,22.315704,0.762316,0.860158,0.865855,Visual Learner
1,4.833850,8.398520,10.821124,0.726826,0.717105,0.675186,Kinesthetic Learner
2,13.764116,18.056675,18.736389,0.555639,0.612328,0.793344,Auditory Learner
3,18.481643,16.984392,23.504387,0.774959,0.700815,0.702778,Solitary Learner
4,13.242961,16.299147,6.678983,0.664808,0.877323,0.814933,Reading/Writing Learner


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the dataset
data = pd.read_csv('learning_preferences.csv')

# Encode the labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Split the dataset into training and testing sets
X = data.drop(columns=['label'])
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM layer
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the RNN model using LSTM
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output classes for the 7 learning types
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model
model.save('learning_preference_model_rnn.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/50
1750/1750 [==============================] - 7s 3ms/step - loss: 0.4061 - accuracy: 0.8465 - val_loss: 0.2270 - val_accuracy: 0.9159
Epoch 2/50
1750/1750 [==============================] - 4s 2ms/step - loss: 0.1971 - accuracy: 0.9260 - val_loss: 0.1640 - val_accuracy: 0.9391
Epoch 3/50
1750/1750 [==============================] - 5s 3ms/step - loss: 0.1723 - accuracy: 0.9358 - val_loss: 0.1522 - val_accuracy: 0.9445
Epoch 4/50
1750/1750 [==============================] - 5s 3ms/step - loss: 0.1617 - accuracy: 0.9393 - val_loss: 0.1536 - val_accuracy: 0.9425
Epoch 5/50
1750/1750 [==============================] - 4s 3ms/step - loss: 0.1580 - accuracy: 0.9419 - val_loss: 0.1465 - val_accuracy: 0.9467
Epoch 6/50
1750/1750 [==============================] - 6s 3ms/step - loss: 0.1514 - accuracy: 0.9435 - val_loss: 0.1724 - val_accuracy: 0.9367
Epoch 7/50
1750/1750 [==============================] - 4s 2ms/step - loss: 0.1490 - accuracy: 0.9451 - val_loss: 0.1436 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


438/438 [==============================] - 1s 2ms/step - loss: 0.1367 - accuracy: 0.9509
Test Accuracy: 95.09%


In [7]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [13]:
import streamlit as st
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('learning_preference_model_rnn.h5')

# Define the label encoder for decoding the predicted labels
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['Auditory Learner', 'Kinesthetic Learner', 'Logical Learner', 'Reading/Writing Learner', 'Social Learner', 'Solitary Learner', 'Visual Learner'])

st.title("📓 QUIZ")
st.subheader("Questions")

# Define multiple sets of questions with correct answers
question_sets = [
    [
        {"type": "audio", "question": "Listen to the audio and choose the correct answer for question 1.1", "options": ["Option 1.1.1", "Option 1.1.2", "Option 1.1.3"], "answer": "Option 1.1.2", "media": "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3"},
        {"type": "picture", "question": "Look at the picture and choose the correct answer for question 1.2", "options": ["Option 1.2.1", "Option 1.2.2", "Option 1.2.3"], "answer": "Option 1.2.1", "media": "https://via.placeholder.com/150"},
        {"type": "paragraph", "question": "Read the paragraph and choose the correct answer for question 1.3", "options": ["Option 1.3.1", "Option 1.3.2", "Option 1.3.3"], "answer": "Option 1.3.3", "media": "This is a sample paragraph for question 1.3."}
    ],
    [
        {"type": "audio", "question": "Listen to the audio and choose the correct answer for question 2.1", "options": ["Option 2.1.1", "Option 2.1.2", "Option 2.1.3"], "answer": "Option 2.1.3", "media": "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3"},
        {"type": "picture", "question": "Look at the picture and choose the correct answer for question 2.2", "options": ["Option 2.2.1", "Option 2.2.2", "Option 2.2.3"], "answer": "Option 2.2.2", "media": "https://via.placeholder.com/150"},
        {"type": "paragraph", "question": "Read the paragraph and choose the correct answer for question 2.3", "options": ["Option 2.3.1", "Option 2.3.2", "Option 2.3.3"], "answer": "Option 2.3.1", "media": "This is a sample paragraph for question 2.3."}
    ]
]

# Initialize session state to keep track of the current question set, start time, and individual times
if 'current_question_set' not in st.session_state:
    st.session_state.current_question_set = 0
    st.session_state.start_time = time.time()
    st.session_state.question_times = []

# Get the current set of questions
questions = question_sets[st.session_state.current_question_set]

# Create a form for the quiz
form = st.form(key="quiz_form")

# Dictionary to store the answers and response times
answers = {}
question_times = st.session_state.question_times

# Loop through the questions and create appropriate input based on type
for i, q in enumerate(questions):
    if len(question_times) <= i:
        question_times.append(time.time())

    if q["type"] == "audio":
        form.audio(q["media"])
    elif q["type"] == "picture":
        form.image(q["media"])
    elif q["type"] == "paragraph":
        form.write(q["media"])

    answers[f"q{i+1}"] = form.radio(q["question"], options=q["options"])

# Submit button for the form
submitted = form.form_submit_button("Submit your answers")

# Display the answers and move to the next set of questions
if submitted:
    end_time = time.time()
    individual_times = [end_time - t for t in question_times]

    correctness_audio = sum(1 for i, q in enumerate(questions) if q["type"] == "audio" and answers[f"q{i+1}"] == q["answer"])
    correctness_picture = sum(1 for i, q in enumerate(questions) if q["type"] == "picture" and answers[f"q{i+1}"] == q["answer"])
    correctness_paragraph = sum(1 for i, q in enumerate(questions) if q["type"] == "paragraph" and answers[f"q{i+1}"] == q["answer"])

    time_audio = sum(individual_times[i] for i, q in enumerate(questions) if q["type"] == "audio")
    time_picture = sum(individual_times[i] for i, q in enumerate(questions) if q["type"] == "picture")
    time_paragraph = sum(individual_times[i] for i, q in enumerate(questions) if q["type"] == "paragraph")

    input_features = np.array([[time_audio, correctness_audio, time_picture, correctness_picture, time_paragraph, correctness_paragraph]])

    # Predict the learning preference
    prediction = model.predict(input_features)
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))

    st.write(f"Response time for audio questions: {time_audio:.2f} seconds")
    st.write(f"Correct answers for audio questions: {correctness_audio}")
    st.write(f"Response time for picture questions: {time_picture:.2f} seconds")
    st.write(f"Correct answers for picture questions: {correctness_picture}")
    st.write(f"Response time for paragraph questions: {time_paragraph:.2f} seconds")
    st.write(f"Correct answers for paragraph questions: {correctness_paragraph}")
    st.write(f"Predicted learning preference: {predicted_label[0]}")

    for i, q in enumerate(questions):
        st.write(f"Your answer for question {i+1}: {answers[f'q{i+1}']} (Correct: {q['answer']})")

    if st.session_state.current_question_set < len(question_sets) - 1:
        st.session_state.current_question_set += 1
        st.session_state.start_time = time.time()  # Reset the start time for the next set
        st.experimental_rerun()  # Rerun the app to load the next set of questions
    else:
        st.write("No more questions available.")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/mnt/data/learning_preference_model.h5')

# Define the label encoder for decoding the predicted labels
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['Auditory Learner', 'Kinesthetic Learner', 'Logical Learner', 'Reading/Writing Learner', 'Social Learner', 'Solitary Learner', 'Visual Learner'])

# Function to predict learning preference
def predict_learning_preference(input_features):
    prediction = model.predict(input_features)
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
    return predicted_label[0]

# Generate sample inputs for testing the model
sample_input_features = np.array([
    [15, 2, 25, 1, 35, 3],  # Example 1: Different response times and correctness scores
    [10, 3, 20, 3, 30, 2],  # Example 2: Different response times and correctness scores
    [5, 1, 15, 1, 25, 1]    # Example 3: Different response times and correctness scores
])

# Predict and display the learning preferences for the sample inputs
for i, sample_input in enumerate(sample_input_features):
    predicted_preference = predict_learning_preference(sample_input.reshape(1, -1))
    print(f"Sample Input {i+1}: {sample_input}")
    print(f"Predicted Learning Preference: {predicted_preference}\n")


In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('learning_preference_model_rnn.h5')

# Define the label encoder for decoding the predicted labels
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['Auditory Learner', 'Kinesthetic Learner', 'Logical Learner', 'Reading/Writing Learner', 'Social Learner', 'Solitary Learner', 'Visual Learner'])

# Function to predict learning preference
def predict_learning_preference(input_features):
    input_features = input_features.reshape((input_features.shape[0], 1, input_features.shape[1]))
    prediction = model.predict(input_features)
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
    return predicted_label[0]

# Generate sample inputs for testing the model
sample_input_features = np.array([
    [1, 2, 2, 10, 35, 3],  # Example 1: Different response times and correctness scores
    [10, 30, 20, 1, 3, 2],  # Example 2: Different response times and correctness scores
    [50, 1, 1, 10, 25, 1]    # Example 3: Different response times and correctness scores
])

# Predict and display the learning preferences for the sample inputs
for i, sample_input in enumerate(sample_input_features):
    predicted_preference = predict_learning_preference(sample_input.reshape(1, -1))
    print(f"Sample Input {i+1}: {sample_input}")
    print(f"Predicted Learning Preference: {predicted_preference}\n")


1/1 [==============================] - 0s 193ms/step
Sample Input 1: [ 1  2  2 10 35  3]
Predicted Learning Preference: Reading/Writing Learner

1/1 [==============================] - 0s 20ms/step
Sample Input 2: [10 30 20  1  3  2]
Predicted Learning Preference: Logical Learner

1/1 [==============================] - 0s 20ms/step
Sample Input 3: [50  1  1 10 25  1]
Predicted Learning Preference: Kinesthetic Learner

